In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import firwin, lfilter
from sklearn.decomposition import FastICA

# Read EEG files from a folder
def read_eeg_files(folder_path):
    if not os.path.exists(folder_path):
        raise ValueError("The folder path does not exist.")
    
    eeg_files = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            eeg_files[filename] = np.genfromtxt(file_path, delimiter=',')
    return eeg_files

# Perform re-referencing
def perform_rereferencing(df):
    average_TP9_TP10 = (df['TP9'] + df['TP10']) / 2
    df['AF7'] -= average_TP9_TP10
    df['AF8'] -= average_TP9_TP10
    return df

#  high_pass_filter function with configurable parameters
def high_pass_filter(eeg_data, fs=256, numtaps=1129, passband_edge=0.75):
    #fir_coefs = firwin(numtaps, passband_edge, nyquist=fs/2, pass_zero='highpass', window='hamming')
    # Corrected line inside high_pass_filter function
    fir_coefs = firwin(numtaps, passband_edge / (0.5 * fs), pass_zero='highpass', window='hamming')

    filtered_eeg = lfilter(fir_coefs, 1.0, eeg_data)
    return filtered_eeg

#  artifact removal using ICA
def remove_artifacts_with_ICA(eeg_data):
    ica = FastICA(n_components=1)
    eeg_data_transformed = ica.fit_transform(eeg_data.values.reshape(-1, 1))
    return pd.Series(eeg_data_transformed.ravel())


# main processing function
def process_eeg_data_modified(folder_path):
    if not os.path.exists(folder_path):
        raise ValueError("The folder path does not exist.")
        
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            if 'Right Aux' in df.columns:
                df.drop(columns=['Right Aux'], inplace=True)
            df = perform_rereferencing(df)
            df['AF7'] = high_pass_filter(df['AF7'], numtaps=1281, passband_edge=1.0)  
            df['AF8'] = high_pass_filter(df['AF8'], numtaps=1281, passband_edge=1.0) 
            #df['AF7'] = remove_artifacts_with_ICA(df['AF7'])
            #df['AF8'] = remove_artifacts_with_ICA(df['AF8'])
            df.dropna(inplace=True)
            new_filename = 'cleaned_' + filename
            new_file_path = os.path.join(folder_path, new_filename)
            df.to_csv(new_file_path, index=False)
            print(f"Processed and saved cleaned data to {new_file_path}")

# your actual folder path
folder_path = '/Users/steven/Desktop/RE_muse_alpha/raw_data/origina_data'
process_eeg_data_modified(folder_path)


In [1]:
import os
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.signal.windows import hamming

# Define the folder paths
folder_path = '/Users/steven/Desktop/RE_muse_alpha/raw_data/cleaned_data_ICA'
output_folder = '/Users/steven/Desktop/RE_muse_alpha/raw_data/PSD_cleaned_ICA'

# Get all files in the directory
all_files = os.listdir(folder_path)

# Filter for CSV files
file_names = [f for f in all_files if f.endswith('.csv')]

# Other parameters
fs = 256
window_length = 4 * fs
overlap = window_length // 2
nperseg = window_length
noverlap = overlap
nfft = 2 * nperseg
window = hamming(nperseg)

# Loop through each file
for file_name in file_names:
    # Dictionary to store the average PSD in the alpha band
    avg_psd_alpha = {'AF7': [], 'AF8': []}
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(os.path.join(folder_path, file_name))
    
    # Extract AF7 and AF8 channel data
    af7_data = df['AF7'].values
    af8_data = df['AF8'].values
    
    # Compute PSD
    freqs, psd_af7 = welch(af7_data, fs, window=window, noverlap=noverlap, nperseg=nperseg, nfft=nfft)
    _, psd_af8 = welch(af8_data, fs, window=window, noverlap=noverlap, nperseg=nperseg, nfft=nfft)
    
    # Define the alpha band
    alpha_low, alpha_high = 8, 13
    
    # Find indices for the alpha band
    idx_alpha = np.where((freqs >= alpha_low) & (freqs <= alpha_high))
    
    # Average PSD for alpha band and store in the dictionary
    avg_psd_alpha['AF7'].append(np.mean(psd_af7[idx_alpha]))
    avg_psd_alpha['AF8'].append(np.mean(psd_af8[idx_alpha]))
    
    # Convert the dictionary to a DataFrame
    output_df = pd.DataFrame(avg_psd_alpha)
    
    # Create a unique output filename for each input file
    output_file = os.path.join(output_folder, f"psd_{file_name}")
    
    # Save to a new CSV file
    output_df.to_csv(output_file, index=True)

In [ ]:
import os
import pandas as pd
import numpy as np

# Define folder paths
input_folder = '/Users/steven/Desktop/RE_muse_alpha/raw_data/PSD_cleaned_ICA'
output_folder = '/Users/steven/Desktop/RE_muse_alpha/raw_data/FAA_cleaned_ICA'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Initialize an empty list to store FAA results
faa_results = []

# Get all files in the directory
file_names = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Loop through each file
for file_name in file_names:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(os.path.join(input_folder, file_name), index_col=0)
    
    # Extract alpha power for AF7 and AF8
    alpha_power_af7 = df.loc[0, 'AF7']
    alpha_power_af8 = df.loc[0, 'AF8']

    # Check for invalid values before applying log
    if alpha_power_af7 <= 0 or alpha_power_af8 <= 0:
        print(f"Skipping {file_name} due to invalid alpha power values.")
        continue
    
    # Calculate FAA
    #faa = np.log(alpha_power_af8) - np.log(alpha_power_af7)
    faa = 10*np.log(alpha_power_af8) - 10*np.log(alpha_power_af7)
    
    # Append result to list
    faa_results.append({'File': file_name, 'FAA': faa})

# Convert the list of dictionaries to a DataFrame
faa_results_df = pd.DataFrame(faa_results)

# Save the FAA results to a single CSV file
output_file_path = os.path.join(output_folder, 'FAA_results.csv')
faa_results_df.to_csv(output_file_path, index=False)
 